In [29]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [42]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_0.5_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_10ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_12ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_13ns_400_ratio_runs'
file_list = 'runlist/LED_1.7V_14ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_15ns_400_ratio_runs'

In [43]:
path_save =""
winfo =[]
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]            
            area = process_data.pusle_area(pulse, st, ed, base)
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'Area': area,     ## area of pulse cal by dynamic range
            })            
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -13]  
        path_save = "outnpy/{}.npy".format(file_tag)

359511it [00:08, 40926.71it/s]


In [44]:
path_save

'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_14ns_400_ratio_run.npy'

In [45]:
df = pd.DataFrame(winfo)
data_array = df.values
np.save(path_save, data_array)